In [16]:
import numpy as np
import time
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [17]:
def boids_step_python(agents, R, ALPHA):
    N = len(agents)
    agents_next = np.zeros_like(agents)
    for i in range(N):
        px, py, vx, vy = agents[i]
        avg_vx, avg_vy = 0.0, 0.0
        neighbor_count = 0
        for j in range(N):
            if i == j:
                continue
            dist_sq = (px - agents[j, 0])**2 + (py - agents[j, 1])**2
            if dist_sq < R**2:
                neighbor_count += 1
                avg_vx += agents[j, 2]
                avg_vy += agents[j, 3]
        
        if neighbor_count > 0:
            avg_vx /= neighbor_count
            avg_vy /= neighbor_count
            vx_next = vx + ALPHA * (avg_vx - vx)
            vy_next = vy + ALPHA * (avg_vy - vy)
        else:
            vx_next, vy_next = vx, vy
            
        px_next = px + vx_next
        py_next = py + vy_next
        agents_next[i] = [px_next, py_next, vx_next, vy_next]
    return agents_next

In [ ]:
# Need to review
def test_alignment_rule():
    print("--- Running Correctness Test for Alignment ---")
    
    R_test = 10.0
    ALPHA_test = 0.5
    
    initial_agents = np.array([
        [0.0, 0.0, 1.0, 0.0],  # Boid 0
        [1.0, 1.0, 0.0, 1.0],  # Boid 1
        [-1.0, 1.0, 0.0, 1.0], # Boid 2
    ], dtype=np.float32)
    
    final_agents = boids_step_python(initial_agents, R_test, ALPHA_test)
    
    expected_boid_0 = np.array([0.5, 0.5, 0.5, 0.5], dtype=np.float32)
    
    actual_boid_0 = final_agents[0]
    
    print(f"Initial Boid 0: {initial_agents[0]}")
    print(f"Expected Boid 0 after 1 step: {expected_boid_0}")
    print(f"Actual Boid 0 after 1 step:   {actual_boid_0}")
    
    assert np.allclose(actual_boid_0, expected_boid_0), "Test Failed!"
    
    print("\n✅ Correctness Test Passed!")

# Run the test
test_alignment_rule()

--- Running Correctness Test for Alignment ---
Initial Boid 0: [0. 0. 1. 0.]
Expected Boid 0 after 1 step: [0.5 0.5 0.5 0.5]
Actual Boid 0 after 1 step:   [0.5 0.5 0.5 0.5]

✅ Correctness Test Passed!


In [25]:
def run_and_visualize_boids(num_agents=150, num_frames=200):
    R_param = 50.0
    ALPHA_param = 0.1
    WIDTH, HEIGHT = 800, 600

    np.random.seed(42)
    agents = np.zeros((num_agents, 4), dtype=np.float32)
    agents[:, 0] = np.random.rand(num_agents) * WIDTH
    agents[:, 1] = np.random.rand(num_agents) * HEIGHT
    
    angles = np.random.rand(num_agents) * 2 * np.pi
    initial_speed = 2.0
    agents[:, 2] = np.cos(angles) * initial_speed
    agents[:, 3] = np.sin(angles) * initial_speed

    fig, ax = plt.subplots(figsize=(10, 7.5))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    
    ax.set_title(f'Boids Simulation (Alignment-only, N={num_agents})')
    ax.set_facecolor('black')
    ax.set_xlim(0, WIDTH)
    ax.set_ylim(0, HEIGHT)
    ax.set_xticks([])
    ax.set_yticks([])

    quiver = ax.quiver(
        agents[:, 0],
        agents[:, 1],
        agents[:, 2],
        agents[:, 3],
        color='cyan',
        headwidth=5,
        scale=50
    )

    def update_frame(frame_number):
        nonlocal agents
        
        agents = boids_step_python(agents, R_param, ALPHA_param)
        
        agents[:, 0] %= WIDTH
        agents[:, 1] %= HEIGHT
        
        quiver.set_offsets(agents[:, :2])
        quiver.set_UVC(agents[:, 2], agents[:, 3])
        
        return quiver,

    animation = FuncAnimation(
        fig,
        update_frame,
        frames=num_frames,
        interval=50,
        blit=True
    )

    plt.close(fig)
    return HTML(animation.to_html5_video())

In [26]:
run_and_visualize_boids(num_agents=250, num_frames=250)

In [12]:
# ==================================================================
# The Performance Benchmark Function
# ==================================================================
def run_benchmark(num_agents, num_steps=10):
    """
    Runs the Boids simulation for a given number of agents and
    measures the average time per step.
    """
    print(f"--- Benchmarking with N = {num_agents} agents ---")
    
    R = 5.0
    ALPHA = 0.05
    WIDTH, HEIGHT = 800.0, 600.0


    np.random.seed(42) 
    agents = np.zeros((num_agents, 4), dtype=np.float32)
    agents[:, 0] = np.random.rand(num_agents) * WIDTH
    agents[:, 1] = np.random.rand(num_agents) * HEIGHT
    angles = np.random.rand(num_agents) * 2 * np.pi
    agents[:, 2] = np.cos(angles)
    agents[:, 3] = np.sin(angles)

    agents = boids_step_python(agents, R, ALPHA)

    # Now, run the main timing loop
    start_time = time.perf_counter()
    for _ in range(num_steps):
        agents = boids_step_python(agents, R, ALPHA)
    end_time = time.perf_counter()

    total_duration = end_time - start_time
    avg_time_per_step = total_duration / num_steps
    
    print(f"Completed {num_steps} steps in {total_duration:.4f} seconds.")
    print(f"Average time per step: {avg_time_per_step * 1000:.2f} ms\n")
    
    return avg_time_per_step

In [13]:
agent_counts = [100, 500, 1000]
    
performance_results = {}

for n in agent_counts:
    avg_time = run_benchmark(n)
    performance_results[n] = avg_time
    
print("--- Python Performance Summary ---")
for n, t in performance_results.items():
    print(f"N = {n:<5} | Avg Time = {t * 1000:.2f} ms/step")

--- Benchmarking with N = 100 agents ---
Completed 10 steps in 0.5116 seconds.
Average time per step: 51.16 ms

--- Benchmarking with N = 500 agents ---
Completed 10 steps in 13.3261 seconds.
Average time per step: 1332.61 ms

--- Benchmarking with N = 1000 agents ---
Completed 10 steps in 55.6921 seconds.
Average time per step: 5569.21 ms

--- Benchmarking with N = 2000 agents ---
Completed 10 steps in 239.3404 seconds.
Average time per step: 23934.04 ms

--- Python Performance Summary ---
N = 100   | Avg Time = 51.16 ms/step
N = 500   | Avg Time = 1332.61 ms/step
N = 1000  | Avg Time = 5569.21 ms/step
N = 2000  | Avg Time = 23934.04 ms/step
